# Get genres

https://developer.spotify.com/documentation/web-api/reference/tracks/get-several-audio-features/

>- `genre` `array of strings` A list of the genres the artist is associated with. For example: "Prog Rock" , "Post-Grunge". (If not yet classified, the array is empty.)
>- `uri`	string	The Spotify URI for the track.

In [ ]:
%run ../credentials/credentials.ipynb
CLIENT_ID = Client_ID
CLIENT_SECRET = Client_Secret

In [3]:
import requests
import pandas as pd
import time as tm
import os
import glob
import json

from time import time

In [ ]:
def auth():
    AUTH_URL = 'https://accounts.spotify.com/api/token'

    # POST
    auth_response = requests.post(AUTH_URL, {
        'grant_type': 'client_credentials',
        'client_id': CLIENT_ID,
        'client_secret': CLIENT_SECRET,
        'Retry-After': 6
    })

    # convert the response to JSON
    auth_response_data = auth_response.json()

    # save the access token
    access_token = auth_response_data['access_token']
    
    headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
    }
    
    return headers

In [ ]:
headers = auth()

BASE_URL = 'https://api.spotify.com/v1/artists?ids='

In [ ]:
track_uri = pd.read_csv('../data-processed/full-data/pid-track-artist.csv')

In [ ]:
track_uri.head()

In [ ]:
track_uri.shape

In [ ]:
artist_uri = list(track_uri.artist_uri.unique())

In [ ]:
BASE_URL + artist_ids

In [ ]:
lim = len(artist_uri)
lim=100
lim_low = 0
step=50
time0 = time()
for i in range(lim_low,lim,step):
    L = i
    if i + step <= lim:
        U = i+step
    else:
        U = lim
    artist_ids = ','.join([artist.replace('spotify:artist:',"") for artist in artist_uri[L:U]])
    #print(L,U)
    r = requests.get(BASE_URL +  artist_ids, headers=headers)     
    if r.status_code!=200:
        print(r.status_code)
    #print('tr',track_ids)
    if r.status_code == 200:
        D={}
        for el in r.json()['artists']:
            for k,v in el.items():
                D[el['uri']] = [g for g in el['genres'] if g!='']       
        with open(f'../data-processed/genres/artist-genre{L}-{U}.json', 'w') as f:
            json.dump(D, f)
    else:
        tm.sleep(30)
        failed.append(track_ids)
        print(r.status_code)
        

In [ ]:
len(D)

# Combine all files 

In [ ]:
D_all={}
path_to_jsonfiles = '../data-processed/genres/'
for file in os.listdir(path_to_jsonfiles):
    with open(f'{path_to_jsonfiles}{file}','r') as fi:
        D_all.update(json.load(fi))

In [ ]:
len(D_all)

In [2]:
#D_all

In [5]:
d = pd.read_csv('../data-processed/full-data/genres_by_artist.csv')

In [6]:
d.head()

,artist_uri,21st century classical,432hz,8-bit,8d,a cappella,aarhus indie,aberdeen indie,abstract,abstract beats,...,zim hip hop,zim urban groove,zimdancehall,zither,zolo,zouglou,zouk,zouk riddim,zurich indie,zydeco
0,spotify:artist:0001ZVMPt41Vwzt1zsmuzp,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,spotify:artist:0001wHqxbF2YYRQxGdbyER,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,spotify:artist:000Dq0VqTZpxOP6jQMscVL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,spotify:artist:000spuc3oKgwYmfg5IE26s,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,spotify:artist:000xagx3GkcunHTFdB4ly0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
d.genre